In [37]:
# des librairies
import numpy
import pandas as pd
import openai
import tiktoken

In [38]:
# importation des donnes
#data source https://www.kaggle.com/competitions/nlp-getting-started
data=pd.read_csv('train (1).csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [39]:
#le nombre de tokens encodés.
def coun_token(text,model_name="cl100k_base"):
    encoding=tiktoken.get_encoding(model_name)
    return len(encoding.encode(text=text))

In [40]:
# cleaning tweet
import re

def clean_tweet_text(text):
    # Supprimer les hashtags
    text=text.lower()
    text = re.sub(r'#\w+', '', text)
    # Supprimer les mentions d'utilisateur
    text = re.sub(r'@\w+', '', text)
    # Supprimer les liens
    text = re.sub(r'http\S+', '', text)
    # Supprimer les symboles de ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace("\n", " ")
    # Retourner le texte nettoyé
    return text

In [41]:
def get_embedding(text):
    #text = text.replace("\n", " ")
    return openai.Embedding.create(input =text, model="text-embedding-ada-002")['data'][0]['embedding']

In [42]:
data['clean_tweet']=data['text'].apply(clean_tweet_text)

In [43]:
data.head()

,id,keyword,location,text,target,clean_tweet
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this may allah fo...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive evacuation orders in cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby as smoke f...


In [44]:
# compter le nombre de tokens
data['num_token']=data['clean_tweet'].apply(coun_token)

In [113]:
# estimation du cout
data.num_token.sum()*0.0004/1000

AttributeError: 'DataFrame' object has no attribute 'num_token'

In [46]:
data=data.sample(frac=0.65,random_state=42)

In [47]:
data=data.loc[:,["clean_tweet",'target']]
data.head()

,clean_tweet,target
2644,so you have a new weapon that can cause unimag...,1
2227,the famp things i do for just got soaked in a...,0
5448,dt rt ûïthe col police can catch a pickpocke...,1
132,aftershock back to school kick off was great i...,0
6845,in response to trauma children of addicts deve...,0


In [48]:
data.shape

(4948, 2)

In [51]:
import os

In [52]:
openai.api_key="KEY"

In [53]:
data['embedding']=data['clean_tweet'].apply(get_embedding)

In [54]:
data.head()

,clean_tweet,target,embedding
2644,so you have a new weapon that can cause unimag...,1,"[-0.01535287220031023, -0.005396141670644283, ..."
2227,the famp things i do for just got soaked in a...,0,"[-0.017056267708539963, -0.004621139261871576,..."
5448,dt rt ûïthe col police can catch a pickpocke...,1,"[-0.0024616981390863657, 0.01179935410618782, ..."
132,aftershock back to school kick off was great i...,0,"[0.009032449685037136, -0.0015162565978243947,..."
6845,in response to trauma children of addicts deve...,0,"[-0.014810529537498951, 0.0043197376653552055,..."


In [55]:
data.to_csv('embedding.csv')

In [59]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

In [60]:
X=list(data["embedding"])
y=data.target

In [61]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [62]:
# Instancier les modèles
rf = RandomForestClassifier(random_state=42)

# Entraîner les modèles
rf.fit(X_train, y_train)

# Évaluer les modèles sur les données de test
rf_score = accuracy_score(y_test, rf.predict(X_test))

# Afficher les scores
print(f"Random Forest score: {rf_score}")


Random Forest score: 0.8121212121212121


In [63]:
print(classification_report(y_test,rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       541
           1       0.89      0.67      0.76       449

    accuracy                           0.81       990
   macro avg       0.83      0.80      0.80       990
weighted avg       0.83      0.81      0.81       990



In [65]:
# Instancier les modèles

#gb = GradientBoostingClassifier(random_state=42)
xgb = XGBClassifier()
lr = LogisticRegression()

# Entraîner les modèles
#gb.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lr.fit(X_train, y_train)


# Évaluer les modèles sur les données de test

#gb_score = accuracy_score(y_test, gb.predict(X_test))
xgb_score = accuracy_score(y_test, xgb.predict(X_test))
lr_score = accuracy_score(y_test, lr.predict(X_test))

# Afficher les scores
#print(f"Gradient Boosting score: {gb_score}")
print(f"XGBoost score: {xgb_score}")
print(f"Logistic Regression score: {lr_score}")

XGBoost score: 0.8313131313131313
Logistic Regression score: 0.8373737373737373


In [66]:
print(classification_report(y_test,lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.88      0.86       541
           1       0.85      0.78      0.81       449

    accuracy                           0.84       990
   macro avg       0.84      0.83      0.83       990
weighted avg       0.84      0.84      0.84       990



In [68]:
from sklearn.model_selection import cross_val_score

# Créer un objet LogisticRegression

lr = LogisticRegression()

# Effectuer une cross-validation à 5 plis et calculer les scores

scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='accuracy')

# Afficher les scores moyens et l'écart-type

print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())
print("Standard deviation:", scores.std())


Cross-validation scores: [0.8270202  0.81944444 0.82954545 0.85208597 0.82553729]
Mean score: 0.8307266725408319
Standard deviation: 0.011185445718753607


In [97]:
# try a 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [98]:
tf.random.set_seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=( 1536,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [99]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [100]:
stop_model=EarlyStopping(patience=5,monitor='val_loss')

In [101]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train.tolist()).reshape(-1, 1)
X_test = np.array(X_train)
y_test = np.array(y_train.tolist()).reshape(-1, 1)

In [102]:
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test,y_test))

Epoch 1/100
124/124 [==============================] - 1s 7ms/step - loss: 0.5389 - accuracy: 0.7557 - val_loss: 0.4419 - val_accuracy: 0.8143
Epoch 2/100
124/124 [==============================] - 1s 5ms/step - loss: 0.4221 - accuracy: 0.8239 - val_loss: 0.3954 - val_accuracy: 0.8295
Epoch 3/100
124/124 [==============================] - 1s 6ms/step - loss: 0.3934 - accuracy: 0.8373 - val_loss: 0.3702 - val_accuracy: 0.8416
Epoch 4/100
124/124 [==============================] - 1s 7ms/step - loss: 0.3829 - accuracy: 0.8408 - val_loss: 0.3586 - val_accuracy: 0.8509
Epoch 5/100
124/124 [==============================] - 1s 7ms/step - loss: 0.3748 - accuracy: 0.8423 - val_loss: 0.3522 - val_accuracy: 0.8555
Epoch 6/100
124/124 [==============================] - 1s 5ms/step - loss: 0.3666 - accuracy: 0.8497 - val_loss: 0.3472 - val_accuracy: 0.8555
Epoch 7/100
124/124 [==============================] - 1s 5ms/step - loss: 0.3616 - accuracy: 0.8471 - val_loss: 0.3438 - val_accuracy: 0.8580

Epoch 58/100
124/124 [==============================] - 1s 5ms/step - loss: 0.1125 - accuracy: 0.9616 - val_loss: 0.0902 - val_accuracy: 0.9793
Epoch 59/100
124/124 [==============================] - 1s 7ms/step - loss: 0.1097 - accuracy: 0.9626 - val_loss: 0.0844 - val_accuracy: 0.9800
Epoch 60/100
124/124 [==============================] - 1s 5ms/step - loss: 0.1060 - accuracy: 0.9644 - val_loss: 0.0847 - val_accuracy: 0.9735
Epoch 61/100
124/124 [==============================] - 1s 5ms/step - loss: 0.1015 - accuracy: 0.9644 - val_loss: 0.0782 - val_accuracy: 0.9813
Epoch 62/100
124/124 [==============================] - 1s 6ms/step - loss: 0.1001 - accuracy: 0.9669 - val_loss: 0.0765 - val_accuracy: 0.9765
Epoch 63/100
124/124 [==============================] - 1s 8ms/step - loss: 0.1026 - accuracy: 0.9656 - val_loss: 0.0731 - val_accuracy: 0.9831
Epoch 64/100
124/124 [==============================] - 1s 6ms/step - loss: 0.0961 - accuracy: 0.9694 - val_loss: 0.0708 - val_accuracy:

In [103]:
model.evaluate(X_test,y_test)

124/124 [==============================] - 0s 2ms/step - loss: 0.0314 - accuracy: 0.9896


[0.0313883051276207, 0.9896412491798401]

In [104]:
print(classification_report(y_test,model.predict(X_test)>0.5))

124/124 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2303
           1       0.99      0.99      0.99      1655

    accuracy                           0.99      3958
   macro avg       0.99      0.99      0.99      3958
weighted avg       0.99      0.99      0.99      3958



In [105]:
from sklearn.manifold import TSNE

In [106]:
# Create a t-SNE model and transform the data
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(X_train)
vis_dims.shape

(3958, 2)